In [2]:
RUN_TEST = True # Do a fast run to check the pipeline is working
RUN_1_BATCH = False # Do a fast run to check the pipeline is working
MAX_EPOCH = 10

RUN_NAME = 'TEST_1' # WandB run name
PROJECT = 'Last_V1' # WandB project name, monitor audio outputs in real time
HARDWARE = 'CPU' # T4 or P100 or CPU
DATA_SAVE = 'data' # Directory to save data
DATA_PATH = r'C:\Users\adhem\Desktop\Music\Custom_VST\data\ARME-Virtuoso-Strings-2.2' # Directory to the dataset
KAGGLE = False

In [4]:
from launch import TrainerWarper
import torch.nn as nn

ImportError: attempted relative import with no known parent package

#### Model Config

In [ ]:
model_cfg = {
    'in_channels': 1,
    'out_channels': 16,
    'channels': [16, 32, 64, 64],
    'factors': [4, 4, 4, 4],
    'res_blocks': 2,
    'activation': nn.SiLU,
    'use_norm': True,
    'num_groups': 4,
    'variational': True,
    'lstm': True,
}

#### Optimizer Config

In [ ]:
optimizer_cfg = {
    'lr': 0.001,
    'betas': (0.9, 0.999),
    'weight_decay': 0,
}

#### Loss Config

In [ ]:
loss_cfg = {
    'L1TemporalLoss':{
        'weight':1,
        'balancer': True,
        'key_output': 'x_hat',
        'key_target': 'x',
        },
    'L2TemporalLoss':{
        'weight':1,
        'balancer': True,
        'key_output': 'x_hat',
        'key_target': 'x',
        },
    'AuralossLoss':{
        'weight':1,
        'balancer': True,
        'key_output': 'x_hat',
        'key_target': 'x',
        "fft_sizes": [32, 128, 512, 2048],  # [32, 128, 512, 2048, 8192, 32768]
        "hop_sizes": [16, 64, 256, 1024],  # [16, 64, 256, 1024, 4096, 16384]
        "win_lengths": [32, 128, 512, 2048],  # [32, 128, 512, 2048, 8192, 32768]
        "w_sc": 0.0,
        "w_phs": 0.0,
        "w_lin_mag": 1.0,
        "w_log_mag": 1.0,
    },
    'KLDivergenceLoss':{
        'weight':1,
        'balancer': False,
        'key_mean': 'z_mean',
        'key_log_std': 'z_log_std',
        },
    'Discriminator':{
        'weight':1,
        'balancer': True,
        #'type':'Spectrogram_VGGStyle',
    },
    'FeatureLoss':{
        'weight':1,
        'balancer': True,
    },
}

#### Data Config

In [ ]:
data_cfg = {
    'path': DATA_PATH,
    'sr': 22050,
    'sample_duration':8, # Seconds
    'latent_compression': 4**4,
    'envelope_detector':{
        'type': 'Max',
        },
    'pitch_detector': {
        'n_fft': 4096,
        'hop_length': 1024,
        'fmin': 75,
        'fmax': 800,
        'threshold': 0.15,
        'win_length': None,  # Defaults to n_fft
        'window': 'hann',
        'center': True,
        'pad_mode': 'reflect'
    },
    'batch_size': 8,
    'num_workers': 2,
}

#### Training Config

In [ ]:
if KAGGLE:
    # Kaggle herdware
    if HARDWARE == 'T4':
        num_gpus = 2
        available_devices = [0,1]
        print('On Kaggle Double T4')
    elif HARDWARE == 'P100':
        num_gpus = 1
        available_devices = [0]
        print('On Kaggle P100')
    else:
        num_gpus = 0
        available_devices = ['CPU']
        print('On Kaggle CPU')
else:
    num_gpus = 0 # No gpu on my computer :(
    available_devices = ['CPU']
    print('On laptop CPU')

accelerator = 'gpu' if num_gpus > 0 else 'cpu'
devices = num_gpus if num_gpus > 0 else 1

In [ ]:
# Training configuration
training_cfg = {
    
    # Hardware
    'accelerator': accelerator,
    'devices': devices,
    'num_nodes': 1,
    'precision': 32,  # Float32
    'deterministic': False,  # Increases training speed for fixed tensor sizes
    'benchmark': True,  # Optimizes for fixed tensor size dataset
    
    # Epochs
    'min_epochs': 4,
    'max_epochs': MAX_EPOCH,
    'max_time': '00:12:00:00',
    'accumulate_grad_batches': 1,
    
    # Callbacks
    'callbacks': [
    ('EarlyStopping', 
     {
        'monitor': 'train_loss',
        'patience': 5, 
        'verbose': True, 
        'mode': 'min'
    }),

    ('ModelCheckpoint', 
     {
        'monitor': 'train_loss', 
        'dirpath': f'{DATA_SAVE}/', 
        'filename': 'best-checkpoint', 
        'save_top_k': 1, 
        'mode': 'min'
    }),
    ],
    
    # Logging / Debug
    'logger': None,  # To be defined later
    'profiler': 'simple',
    'fast_dev_run': RUN_TEST,
    'limit_train_batches': 1 if RUN_1_BATCH else None,
    'enable_checkpointing': True,
    'barebones': False,
}

#### Wandb

In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger

all_config = {
    'TRAINING': training_cfg,
    'MODEL': model_cfg,
    'DATA': data_cfg,
    'TRAINER': training_cfg,
}

training_cfg['logger'] = None if (RUN_TEST or RUN_1_BATCH) else WandbLogger(project=PROJECT, name=RUN_NAME, config=all_config)

In [ ]:
TrainerWarper(
    model_cfg,
    data_cfg,
    optimizer_cfg,
    loss_cfg,
    training_cfg,):
